In [3]:
import pandas as pd

In [45]:
df = pd.read_csv("./data/accepted_2007_to_2018Q4.csv")

C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_22824\2294106053.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/accepted_2007_to_2018Q4.csv")


In [61]:
columns = df.columns
for c in columns:
    print(c)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
to

In [64]:
data = df[[
  "loan_amnt", "term", "int_rate", "installment", "purpose",
  "emp_length", "home_ownership", "annual_inc", "dti",
  "fico_range_low", "fico_range_high",
  "inq_last_6mths", "delinq_2yrs", "earliest_cr_line",
  "open_acc", "pub_rec", "revol_bal", "revol_util", "total_acc", "loan_status", "issue_d"
]]

In [65]:
default_map = {'Fully Paid': 0, 'Current': 0, 'Charged Off': 1, 'In Grace Period': 0,
       'Late (31-120 days)': 1, 'Late (16-30 days)': 0, 'Default': 1,
       'Does not meet the credit policy. Status:Fully Paid': 0,
       'Does not meet the credit policy. Status:Charged Off': 1}

data["loan_risk"] = data["loan_status"].map(default_map)
data = data.dropna(subset=["loan_risk"])
data["loan_risk"] = data["loan_risk"].astype(int)
# 1: High Risk
# 0: Low Risk

C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_22824\4091550644.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["loan_risk"] = data["loan_status"].map(default_map)


In [66]:
missing = data.isnull().mean().sort_values(ascending=False)
print(missing[missing > 0])


emp_length          0.064984
revol_util          0.000797
dti                 0.000757
inq_last_6mths      0.000013
delinq_2yrs         0.000013
total_acc           0.000013
open_acc            0.000013
pub_rec             0.000013
earliest_cr_line    0.000013
annual_inc          0.000002
dtype: float64


In [67]:
data["term"]

0           36 months
1           36 months
2           60 months
3           60 months
4           60 months
              ...    
2260694     60 months
2260695     60 months
2260696     60 months
2260697     60 months
2260698     60 months
Name: term, Length: 2260668, dtype: object

In [68]:
# loan_amnt
data["term"] = data['term'].str.extract('(\d+)').astype(float)  # term
# int_rate
# installment
# purpose
data['emp_length'] = data['emp_length'].fillna("Missing").astype('category')    # emp_length
# home_ownership
# annual_inc
data['installment_to_income'] = data['installment'] / (data['annual_inc'] + 1) # installment_to_income; Ratio of installments to income
# dti
data["fico_score"] = (data["fico_range_high"] + data["fico_range_low"]) / 2   # fico_score <- fico_range_low , fico_range_high; Average
# inq_last_6mths
# delinq_2yrs
data["earliest_cr_line"] = pd.to_datetime(data["earliest_cr_line"], errors="coerce")  # earliest_cr_line
data["credit_age"] = 2025 - data["earliest_cr_line"].dt.year  # credit_age; Age in 2025
# open_acc
# pub_rec
# revol_bal
#data["revol_util"] = data["revol_util"].str.rstrip("%").astype(float)    # revol_util
# total_acc

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_22824\2189944155.py:2: SyntaxWarning: invalid escape sequence '\d'
  data["term"] = data['term'].str.extract('(\d+)').astype(float)  # term
C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_22824\2189944155.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["earliest_cr_line"] = pd.to_datetime(data["earliest_cr_line"], errors="coerce")  # earliest_cr_line


In [69]:
data = data.dropna(how="all")

In [70]:
# Make emp_length numeric
emp_map = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
    'Missing': -1
}
data['emp_length'] = data['emp_length'].map(emp_map)


In [71]:
print(data.isnull().mean().sort_values(ascending=False).head(10))


revol_util               0.000797
dti                      0.000757
inq_last_6mths           0.000013
earliest_cr_line         0.000013
total_acc                0.000013
delinq_2yrs              0.000013
open_acc                 0.000013
credit_age               0.000013
pub_rec                  0.000013
installment_to_income    0.000002
dtype: float64


In [72]:
for col in ["purpose", "home_ownership"]:
    data[col] = data[col].fillna("Missing")
    data[col] = data[col].astype("category")

In [73]:
drop_cols = [
    'id', 'member_id', 'url', 'desc', 'earliest_cr_line',  # already engineered
    'fico_range_low', 'fico_range_high',  # used in fico_score
    'emp_title'  # noisy text, optional
    ,"loan_status"
]
data = data.drop(columns=drop_cols, errors='ignore')


In [75]:
data["issue_d"] = pd.to_datetime(data['issue_d'], format='%b-%Y', errors='coerce')
data['issue_month'] = data['issue_d'].dt.to_period('M')

In [77]:
# Now add macroeconomic factors
indicators = {
    'FEDFUNDS': 'fed_funds_rate',
    'UNRATE': 'unemployment_rate',
    'CPIAUCSL': 'cpi',
    'GDPC1': 'real_gdp',  # quarterly
    'TDSP': 'debt_service_ratio'
}

import pandas_datareader.data as web

start_date = "2007-01-01"
end_date = "2018-12-31"

macro_frames = []
for fred_code, col_name in indicators.items():
    df = web.DataReader(fred_code, 'fred', start_date, end_date).reset_index()
    df.rename(columns={fred_code: col_name}, inplace=True)
    macro_frames.append(df)

In [81]:
macro = macro_frames[0]
for df in macro_frames[1:]:
    macro = pd.merge(macro, df, on='DATE', how='outer')
macro

,DATE,fed_funds_rate,unemployment_rate,cpi,real_gdp,debt_service_ratio
0,2007-01-01,5.25,4.6,203.437,16611.690,15.526607
1,2007-02-01,5.26,4.5,204.226,NaN,NaN
2,2007-03-01,5.26,4.4,205.288,NaN,NaN
3,2007-04-01,5.25,4.5,205.904,16713.314,15.632510
4,2007-05-01,5.25,4.4,206.755,NaN,NaN
...,...,...,...,...,...,...
139,2018-08-01,1.91,3.8,251.663,NaN,NaN
140,2018-09-01,1.95,3.7,252.182,NaN,NaN
141,2018-10-01,2.19,3.8,252.772,20304.874,11.666344
142,2018-11-01,2.20,3.8,252.594,NaN,NaN


In [83]:
macro = macro.sort_values('DATE').fillna(method='ffill') # Fill with quarterly rates
macro['issue_month'] = macro['DATE'].dt.to_period('M')

C:\Users\Jayden Spurgiasz\AppData\Local\Temp\ipykernel_22824\303126879.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  macro = macro.sort_values('DATE').fillna(method='ffill') # Fill with quarterly rates


In [85]:
finalData = data.merge(macro.drop(columns=["DATE"]), on="issue_month", how="left")

In [4]:
finalData = pd.read_csv("./data/curData.csv")

In [5]:
finalData.columns

Index(['Unnamed: 0', 'loan_amnt', 'term', 'int_rate', 'installment', 'purpose',
       'emp_length', 'home_ownership', 'annual_inc', 'dti', 'inq_last_6mths',
       'delinq_2yrs', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'issue_d', 'loan_risk', 'installment_to_income',
       'fico_score', 'credit_age', 'issue_month', 'fed_funds_rate',
       'unemployment_rate', 'cpi', 'real_gdp', 'debt_service_ratio'],
      dtype='object')

In [8]:
finalData["home_ownership"] = finalData["home_ownership"].astype("category")

In [6]:
finalData["purpose"] = finalData["purpose"].astype("category")

In [14]:
for col in finalData.columns:
    if finalData[col].dtype == "object":
        print(finalData[col])

0          2015-12-01
1          2015-12-01
2          2015-12-01
3          2015-12-01
4          2015-12-01
              ...    
2260663    2016-10-01
2260664    2016-10-01
2260665    2016-10-01
2260666    2016-10-01
2260667    2016-10-01
Name: issue_d, Length: 2260668, dtype: object
0          2015-12
1          2015-12
2          2015-12
3          2015-12
4          2015-12
            ...   
2260663    2016-10
2260664    2016-10
2260665    2016-10
2260666    2016-10
2260667    2016-10
Name: issue_month, Length: 2260668, dtype: object


In [15]:
finalData.drop(columns=["issue_d", "issue_month"])

,Unnamed: 0,loan_amnt,term,int_rate,installment,purpose,emp_length,home_ownership,annual_inc,dti,...,total_acc,loan_risk,installment_to_income,fico_score,credit_age,fed_funds_rate,unemployment_rate,cpi,real_gdp,debt_service_ratio
0,0,3600.0,36.0,13.99,123.03,debt_consolidation,10,MORTGAGE,55000.0,5.91,...,13.0,0,0.002237,677.0,22.0,0.24,5.0,237.761,18892.206,11.738945
1,1,24700.0,36.0,11.99,820.28,small_business,10,MORTGAGE,65000.0,16.06,...,38.0,0,0.012619,717.0,26.0,0.24,5.0,237.761,18892.206,11.738945
2,2,20000.0,60.0,10.78,432.66,home_improvement,10,MORTGAGE,63000.0,10.78,...,18.0,0,0.006868,697.0,25.0,0.24,5.0,237.761,18892.206,11.738945
3,3,35000.0,60.0,14.85,829.90,debt_consolidation,10,MORTGAGE,110000.0,17.06,...,17.0,0,0.007544,787.0,17.0,0.24,5.0,237.761,18892.206,11.738945
4,4,10400.0,60.0,22.45,289.91,major_purchase,3,MORTGAGE,104433.0,25.37,...,35.0,0,0.002776,697.0,27.0,0.24,5.0,237.761,18892.206,11.738945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260663,2260663,24000.0,60.0,12.79,543.50,home_improvement,7,MORTGAGE,95000.0,19.61,...,54.0,0,0.005721,667.0,26.0,0.40,4.9,241.741,19304.352,11.865783
2260664,2260664,24000.0,60.0,10.49,515.74,debt_consolidation,10,MORTGAGE,108000.0,34.94,...,58.0,0,0.004775,697.0,34.0,0.40,4.9,241.741,19304.352,11.865783
2260665,2260665,40000.0,60.0,10.49,859.56,debt_consolidation,9,MORTGAGE,227000.0,12.75,...,37.0,0,0.003787,707.0,30.0,0.40,4.9,241.741,19304.352,11.865783
2260666,2260666,24000.0,60.0,14.49,564.56,debt_consolidation,6,RENT,110000.0,18.30,...,31.0,1,0.005132,662.0,26.0,0.40,4.9,241.741,19304.352,11.865783


In [16]:
finalData.to_csv("./data/curData.csv")

In [1]:
len(finalData)

NameError: name 'finalData' is not defined